# Generic Synthetic Trajectory Simulation and Predictive System

Copyright 2025, Denis Rothman

This notebook demonstrates a synthetic trajectory simulation and predictive system for human mobility. It generates random movement data across a predefined grid, flags certain points as missing, and then uses a generative AI model to predict and replace those missing values.

---

**Table of Contents**  
1. **Importing Libraries and Setup**  
2. **Grid Definition and Parameters**  
3. **Trajectory Generation**  
4. **Injecting Missing Steps**  
5. **Visualization of Trajectories**  
6. **Preparing System Messages for AI Prediction**  
7. **Inference and Replacement of Missing Steps**  

**Note:** This educational notebook implemented with an OpenAI GenAI model is based on Tang, P., Yang, C., Xing, T., Xu, X., Jiang, R., and Sezaki, K. (2024). Instruction-Tuning Llama-3-8B Excels in City-Scale Mobility Prediction.https://arxiv.org/abs/2410.23692


# Setting up the environment

This notebook was developed in Google Colab. Colab includes many pre-installed libraries and sets `/content/` as the default directory, meaning you can access files directly by their filename if you wish (e.g., `filename` instead of needing to specify `/content/filename`). This differs from local environments, where you'll often need to install libraries or specify full file paths.

## File downloading script

grequests contains a script to download files from the repository

In [ ]:
!curl -L https://raw.githubusercontent.com/uCertifyLLC/Building-Business-Ready-Generative-AI-Systems/main/commons/grequests.py --output grequests.py

### Installing OpenAI

In [ ]:
from grequests import download
download("commons","requirements01.py")
download("commons","openai_setup.py")
download("commons","reason.py")
download("commons","machine_learning.py")

Downloaded 'requirements01.py' successfully.
Downloaded 'openai_setup.py' successfully.
Downloaded 'reason.py' successfully.
Downloaded 'machine_learning.py' successfully.


In [ ]:
# Run the setup script to install and import dependencies
%run requirements01

Uninstalling 'openai'...
Installing 'openai' version 1.57.1...
'openai' version 1.57.1 is installed.


#### Initializing the OpenAI API key



In [ ]:
google_secrets=True #activates Google secrets in Google Colab
if google_secrets==True:
  import openai_setup
  openai_setup.initialize_openai_api()

OpenAI API key initialized successfully.


In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the API_KEY
  import os
  #API_KEY=[YOUR API_KEY]
  #os.environ['OPENAI_API_KEY'] = API_KEY
  #openai.api_key = os.getenv("OPENAI_API_KEY")
  #print("OpenAI API key initialized successfully.")

#### Importing the API call function

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import make_openai_api_call
from reason import make_openai_reasoning_call

## Chain of Thought(COT)

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import chain_of_thought_reasoning
from reason import memory_reasoning_thread # import memory reasoning thread96

In [ ]:
# AI agent : the messages and prompts for memory agent tasks
download("commons","cot_messages_c6.py") # downloaded messages and prompts

Downloaded 'cot_messages_c6.py' successfully.


### Generic Synthetic Trajectory Simulation

In [ ]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image as PILImage

def create_grid_with_trajectory(grid_size=200, num_points=50, missing_count=5):
    grid = np.zeros((grid_size, grid_size), dtype=int)
    trajectory = []

    x = random.randint(0, grid_size - 1)
    y = random.randint(0, grid_size - 1)
    day = random.randint(1, 365)
    timeslot = random.randint(0, 47)

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    current_dir_index = random.randint(0, 3)

    turn_weights = {-1: 0.15, 0: 0.70, 1: 0.15}

    for _ in range(num_points):
        turn = random.choices(list(turn_weights.keys()), weights=list(turn_weights.values()))[0]
        current_dir_index = (current_dir_index + turn) % len(directions)
        dx, dy = directions[current_dir_index]
        new_x = x + dx
        new_y = y + dy

        if not (0 <= new_x < grid_size and 0 <= new_y < grid_size):
            valid_indices = [idx for idx, (dx_temp, dy_temp) in enumerate(directions) if 0 <= x + dx_temp < grid_size and 0 <= y + dy_temp < grid_size]
            if valid_indices:
                current_dir_index = random.choice(valid_indices)
                dx, dy = directions[current_dir_index]
                new_x = x + dx
                new_y = y + dy
            else:
                new_x, new_y = x, y

        x, y = new_x, new_y
        trajectory.append((day, timeslot, x, y))
        grid[x, y] = 1
        timeslot = (timeslot + random.randint(1, 3)) % 48

    missing_indices = random.sample(range(len(trajectory)), min(missing_count, len(trajectory)))
    for idx in missing_indices:
        d, t, _, _ = trajectory[idx]
        trajectory[idx] = (d, t, 999, 999)

    x_coords = [x if x != 999 else np.nan for _, _, x, y in trajectory]
    y_coords = [y if y != 999 else np.nan for _, _, x, y in trajectory]

    plt.figure(figsize=(8, 8))
    plt.plot(x_coords, y_coords, marker='o', linestyle='-', color='blue', label="Agent Trajectory")

    valid_indices = [i for i, (xx, yy) in enumerate(zip(x_coords, y_coords)) if not (np.isnan(xx) or np.isnan(yy))]
    if len(valid_indices) > 1:
        valid_x = [x_coords[i] for i in valid_indices]
        valid_y = [y_coords[i] for i in valid_indices]
        dx = np.diff(valid_x)
        dy = np.diff(valid_y)
        plt.quiver(valid_x[:-1], valid_y[:-1], dx, dy, angles='xy', scale_units='xy', scale=1, color='red', width=0.005, label="Direction")

    for (xx, yy) in zip(x_coords, y_coords):
        if not np.isnan(xx) and not np.isnan(yy):
            plt.plot(xx, yy, marker='D', markersize=10, color='green', label='Start')
            break

    for i, (d, t, x, y) in enumerate(trajectory):
        if x == 999 and y == 999:
            prev_valid = next_valid = None
            for j in range(i - 1, -1, -1):
                _, _, xp, yp = trajectory[j]
                if xp != 999 and yp != 999:
                    prev_valid = (xp, yp)
                    break
            for j in range(i + 1, len(trajectory)):
                _, _, xp, yp = trajectory[j]
                if xp != 999 and yp != 999:
                    next_valid = (xp, yp)
                    break
            marker_x, marker_y = prev_valid if prev_valid else next_valid if next_valid else (None, None)
            if marker_x is not None:
                plt.plot(marker_x, marker_y, marker='x', markersize=10, color='magenta', label='Missing Data' if i == missing_indices[0] else "")

    plt.title("Agent Trajectory with Direction Arrows and Missing Data")
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")
    plt.grid(True)
    plt.legend()
    plt.savefig("mobility.png")
    plt.close()

    return grid, trajectory

In [ ]:
import json
import os
import textwrap
from PIL import Image as PILImage

def handle_mobility_orchestrator(muser_message1, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b):
    grid, trajectory = create_grid_with_trajectory(grid_size=200, num_points=50, missing_count=5)

    trajectory_json = json.dumps({"trajectory": trajectory}, indent=2)
    #print("Trajectory Data (JSON):\n", trajectory_json)
    muser_message = f"{muser_message1}\n\nHere is the trajectory data:\n{trajectory_json}"

    reasoning_steps = reason.mobility_agent_reasoning_thread(
        muser_message, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b
    )

    reasoning_steps.insert(0, ("Generated Trajectory Data:", trajectory))

    return reasoning_steps

### Generative Predictive Engine

In [ ]:
def handle_mobility(user_message):
    from cot_messages_c6 import msystem_message_s1, mgeneration, mimcontent4,muser_message1
    mimcontent4b=mimcontent4
    #call Generic Synthetic Trajectory Simulation and Predictive System
    reasoning_steps = handle_mobility_orchestrator(muser_message1, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b)
    return reasoning_steps

In [ ]:
from PIL import Image as PILImage
import os
user_message="Check the delivery path"
output=handle_mobility(user_message)
# Display mobility.png if it exists and the "Mobility" instruction is selected
if os.path.exists("mobility.png"):
  original_image = PILImage.open("mobility.png")
  display(original_image)
print(output)

In [ ]:
def transform_openai_output(output):
    """
    Takes the 'output' (a list/tuple returned by OpenAI) and transforms
    it into a nicely formatted multiline string.
    """
    lines = []
    for item in output:
        # Case 1: If we get a tuple with two elements:
        #         (label_string, list_of_tuples or other data)
        if isinstance(item, tuple) and len(item) == 2:
            label, content = item
            # Print the label
            lines.append(str(label).strip())
            # If the second element is a list of tuples, format each
            if isinstance(content, list):
                for row in content:
                    lines.append("  " + str(row))
            else:
                # Otherwise just add it in a readable way
                lines.append(str(content).strip())

        # Case 2: Otherwise, just handle it as a string
        else:
            lines.append(str(item).strip())

    # Join all lines into one neatly formatted string
    return "\n".join(lines)



pretty_response = transform_openai_output(output)
print(pretty_response)


Generated Trajectory Data:
  (211, 32, 197, 65)
  (211, 35, 999, 999)
  (211, 36, 197, 63)
  (211, 39, 196, 63)
  (211, 41, 195, 63)
  (211, 43, 194, 63)
  (211, 45, 193, 63)
  (211, 46, 192, 63)
  (211, 1, 191, 63)
  (211, 2, 999, 999)
  (211, 4, 999, 999)
  (211, 5, 190, 63)
  (211, 7, 190, 64)
  (211, 9, 190, 65)
  (211, 10, 190, 66)
  (211, 12, 190, 67)
  (211, 14, 190, 68)
  (211, 16, 190, 69)
  (211, 18, 190, 70)
  (211, 19, 191, 70)
  (211, 22, 192, 70)
  (211, 23, 999, 999)
  (211, 26, 192, 68)
  (211, 29, 192, 67)
  (211, 30, 191, 67)
  (211, 32, 191, 68)
  (211, 35, 191, 69)
  (211, 36, 191, 70)
  (211, 39, 191, 71)
  (211, 41, 192, 71)
  (211, 44, 192, 70)
  (211, 46, 192, 69)
  (211, 1, 192, 68)
  (211, 4, 192, 67)
  (211, 5, 191, 67)
  (211, 6, 191, 68)
  (211, 9, 191, 69)
  (211, 10, 191, 70)
  (211, 12, 191, 71)
  (211, 14, 192, 71)
  (211, 17, 999, 999)
  (211, 20, 192, 73)
  (211, 23, 192, 74)
  (211, 24, 192, 75)
  (211, 25, 193, 75)
  (211, 26, 194, 75)
  (211, 27, 1